In [1]:
%run ./read_file.ipynb 

:: loading settings :: url = jar:file:/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ec2-user/.ivy2/cache
The jars for the packages stored in: /home/ec2-user/.ivy2/jars
graphframes#graphframes added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-ba83b9fb-e18c-4119-8c8f-87e74a869269;1.0
	confs: [default]
	found graphframes#graphframes;0.8.2-spark3.2-s_2.12 in spark-packages
	found org.slf4j#slf4j-api;1.7.16 in central
:: resolution report :: resolve 234ms :: artifacts dl 5ms
	:: modules in use:
	graphframes#graphframes;0.8.2-spark3.2-s_2.12 from spark-packages in [default]
	org.slf4j#slf4j-api;1.7.16 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	-----------------------------

23/03/10 15:16:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/10 15:16:59 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).
23/03/10 15:17:00 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [19]:
def build_adjaceny(df, dataset = 'ecar'):
    
    start_time = time.time()
    
    if dataset == 'labels':
        df = df.sort(col("ts2").asc())
    else : 
        #first step sort the data and check
        df = df.sort(col("timestamp").asc())
    
    #now get overlapping connections

    #build collections of each type of ID
    #TODO make this one select statement
    col_objID = df.select("objectID").distinct().collect()
    col_actID = df.select("actorID").distinct().collect()
    
    print(len(col_actID))

    #put these columns into a dictionary and turn into a spark data frame
    data = [{"actorIDs" : [row.actorID for row in col_actID], \
                   "objectIDs" : [row.objectID for row in col_objID]}]

    id_intersect = spark.createDataFrame(data)

    #compare ID pools to find actIDs that are nodes
    id_intersect = id_intersect.withColumn("id_int", \
                                           array_intersect(col("actorIDs"),col("objectIDs")))

    #now build a list of the nodal actIDs and print the length
    actIDs = [row.id_int for row in id_intersect.select("id_int").collect()][0]
    
    print("length actIDs: "+str(len(actIDs)))
    print("elapsed time: "+ str(time.time() - start_time))
    #create ref dict
    dict_node_edge_events = {}
    count = 0 
    
    #for all actors in actIDs
    for aID in actIDs: 
        
        #I think we can pull parent time here and record this in the dictionary somehow
        birth_event = df.where((col("objectID") == aID) \
                               & (col("actorID")!= aID)).limit(1).collect()
        
        if len(birth_event) > 0: 
            str_first_id = birth_event[0]['id']

            #now add this to the dictionary as a key
            first_event = str(aID+':'  
                              +str(birth_event[0]['timestamp'])+":" 
                              +birth_event[0]['object']+":" 
                              +birth_event[0]['parent_image_path']+":" 
                              +birth_event[0]['image_path']+':USER_placeholder')
            
            dict_node_edge_events[first_event] = []

            #now add all the events where the actorID is present but not the objectID, group by unique object ID
            #and pull out the first event - and I checked and believe the sorting holds through grouping
            col_select_children_events = df.select('objectID','id')\
            .where((col("actorID") == aID) & \
                     (col("objectID") != aID)).groupBy("objectID").agg(first("id")).collect() 

            #add children objects into dictioinary 
            dict_node_edge_events[first_event] = [str(row["objectID"]+':'+row["first(id)"]) for row in col_select_children_events]
        else:
            count += 1
    
    print("no birth event for " + str(count)+" events.")
    
    # create a list of tuples from the dictionary
    data_list = list(dict_node_edge_events.items())

    # convert the list of tuples to a PySpark RDD
    data_rdd = spark.sparkContext.parallelize(data_list)

    # create a PySpark DataFrame from the RDD
    df_final = data_rdd.toDF()
    
    #explode list into column, keeping position for some sort of rank just in case
    df_final_exp = df_final.select("_1",posexplode("_2"))
    df_final_exp_col = df_final_exp.selectExpr("_1 as parent", "pos as order", "col as child")
    
    print("elapsed time 2: "+ str(time.time() - start_time))
    return df_final_exp_col

In [3]:
df_labels = readCheckpoint_bcm('labels')
#create a timestamp column and drop the old timestamp - does not appear necessary
df_labels = df_labels.withColumn('ts2', to_timestamp(col('timestamp'))).drop('timestamp')
print('event count' + str(df_labels.count()))
#check new col data types
df_labels.printSchema()

 3:17PM UTC on Mar 10, 2023 --- read and cache time: 1.7642974853515625e-05 seconds ---


event count292367
root
 |-- hostname: string (nullable = true)
 |-- id: string (nullable = true)
 |-- objectID: string (nullable = true)
 |-- actorID: string (nullable = true)
 |-- object: string (nullable = true)
 |-- action: string (nullable = true)
 |-- ts2: timestamp (nullable = true)



In [6]:
#what hosts are in malicious events
col_hosts = df_labels.select('hostname').distinct().collect()
for row in col_hosts:
    print(row.hostname)

SysClient0559.systemia.com
SysClient0255.systemia.com
SysClient0205.systemia.com
SysClient0503.systemia.com
SysClient0355.systemia.com
SysClient0104.systemia.com
SysClient0874.systemia.com
SysClient0955.systemia.com
SysClient0201.systemia.com
SysClient0402.systemia.com
SysClient0660.systemia.com
SysClient0321.systemia.com
SysClient0609.systemia.com
SysClient0771.systemia.com
SysClient0462.systemia.com
SysClient0419.systemia.com
SysClient0010.systemia.com
SysClient0069.systemia.com
SysClient0358.systemia.com
SysClient0203.systemia.com
SysClient0618.systemia.com
SysClient0851.systemia.com
SysClient0501.systemia.com
SysClient0811.systemia.com
SysClient0170.systemia.com
SysClient0351.systemia.com
SysClient0051.systemia.com


In [8]:
#work on extracting parent time and subtracting from child time
#get actIDs
col_objID = df_labels.select("objectID").distinct().collect()
col_actID = df_labels.select("actorID").distinct().collect()

#put these columns into a dictionary and turn into a spark data frame
data = [{"actorIDs" : [row.actorID for row in col_actID], \
               "objectIDs" : [row.objectID for row in col_objID]}]

id_intersect = spark.createDataFrame(data)

#compare ID pools to find actIDs that are nodes
id_intersect = id_intersect.withColumn("id_int", \
                                       array_intersect(col("actorIDs"),col("objectIDs")))

#now build a list of the nodal actIDs and print the length
actIDs = [row.id_int for row in id_intersect.select("id_int").collect()][0]

150


In [11]:
#now pull time and child_event for first actor
first_act = actIDs[0]
birth_event = df_labels.where((col("objectID") == first_act) \
                        & (col("actorID")!= first_act)).limit(1).collect()
print(birth_event)
#great so all the elements are in there, what does the string of the timestamp look like
#it looks fine as a timestampe just turn it into a ts column in the adjaceny matrix

[Row(hostname='SysClient0462.systemia.com', id='e15537fe-fdb0-49d4-a1fd-de34e8215617', objectID='d5846376-9596-4283-8505-8e6112abf1f2', actorID='bf4681e3-9ed7-4824-8612-b478b58a4347', object='PROCESS', action='CREATE', ts2=datetime.datetime(2019, 9, 23, 18, 44, 56, 834000))]


In [21]:
#load in size medium
df_epm = readCheckpoint(size='medium')
list_mal_hosts = ['SysClient0559.systemia.com',
'SysClient0255.systemia.com',
'SysClient0205.systemia.com',
'SysClient0503.systemia.com',
'SysClient0355.systemia.com',
'SysClient0104.systemia.com',
'SysClient0874.systemia.com',
'SysClient0955.systemia.com',
'SysClient0201.systemia.com',
'SysClient0402.systemia.com',
'SysClient0660.systemia.com',
'SysClient0321.systemia.com',
'SysClient0609.systemia.com',
'SysClient0771.systemia.com',
'SysClient0462.systemia.com',
'SysClient0419.systemia.com',
'SysClient0010.systemia.com',
'SysClient0069.systemia.com',
'SysClient0358.systemia.com',
'SysClient0203.systemia.com',
'SysClient0618.systemia.com',
'SysClient0851.systemia.com',
'SysClient0501.systemia.com',
'SysClient0811.systemia.com',
'SysClient0170.systemia.com',
'SysClient0351.systemia.com',
'SysClient0051.systemia.com']
df_epm = df_epm.filter(col("hostname").isin(list_mal_hosts))

# Create a new dataframe with distinct objectIDs to identify malcious ObjectIds
df_malcious_objectIDs = df_labels.select('id').distinct()

# Create a new column called 'malicious' in df_ecar to label malicious records
df_epm = df_epm.withColumn('malicious', when(col('id').isin(df_malcious_objectIDs.rdd.flatMap(lambda x: x).collect()), 1).otherwise(0))

#first let's pull a small section of the benign
df_epm_benign = df_epm.where(col('malicious') == 0).limit(300000)
df_epm_mal = df_epm.where(col('malicious') == 1).limit(10000)


 5:27PM UTC on Mar 10, 2023 --- read and cache time: 0.4163367748260498 seconds ---


In [22]:
df_samp = build_adjaceny(df_epm_mal)

17
length actIDs: 4
elapsed time: 331.3237373828888


23/03/10 17:42:55 ERROR Executor: Exception in task 28.0 in stage 46.0 (TID 527)
java.lang.OutOfMemoryError: Java heap space
	at java.lang.reflect.Array.newInstance(Array.java:75)
	at java.io.ObjectInputStream.readArray(ObjectInputStream.java:2082)
	at java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1656)
	at java.io.ObjectInputStream.readArray(ObjectInputStream.java:2118)
	at java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1656)
	at java.io.ObjectInputStream.defaultReadFields(ObjectInputStream.java:2430)
	at java.io.ObjectInputStream.readSerialData(ObjectInputStream.java:2354)
	at java.io.ObjectInputStream.readOrdinaryObject(ObjectInputStream.java:2212)
	at java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1668)
	at java.io.ObjectInputStream.readObject(ObjectInputStream.java:502)
	at java.io.ObjectInputStream.readObject(ObjectInputStream.java:460)
	at org.apache.spark.serializer.JavaDeserializationStream.readObject(JavaSerializer.scala:87)
	at org.

Py4JJavaError: An error occurred while calling o292587.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 28 in stage 46.0 failed 1 times, most recent failure: Lost task 28.0 in stage 46.0 (TID 527) (ip-172-16-62-81.us-west-2.compute.internal executor driver): java.lang.OutOfMemoryError: Java heap space
	at java.lang.reflect.Array.newInstance(Array.java:75)
	at java.io.ObjectInputStream.readArray(ObjectInputStream.java:2082)
	at java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1656)
	at java.io.ObjectInputStream.readArray(ObjectInputStream.java:2118)
	at java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1656)
	at java.io.ObjectInputStream.defaultReadFields(ObjectInputStream.java:2430)
	at java.io.ObjectInputStream.readSerialData(ObjectInputStream.java:2354)
	at java.io.ObjectInputStream.readOrdinaryObject(ObjectInputStream.java:2212)
	at java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1668)
	at java.io.ObjectIn

----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 54654)
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/socketserver.py", line 747, in __init__
    self.handle()
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/accumulators.py", line 281, in handle
    poll(accum_updates)
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/accumulators.

In [8]:
df_labels_adj.count()

71350

In [10]:
df_labels_adj.select('parent').distinct().count()

90

In [11]:
df_labels_adj.select('child').distinct().count()

71350